In [1]:
import pandas as pd

In [13]:
# df = pd.read_csv('dataset/movies_metadata_ready_big.csv', encoding='utf-8')
df = pd.read_csv('dataset/movies_all_data.csv', encoding='utf-8')


df = df.dropna()
df.shape

(20760, 19)

In [14]:
df.columns

Index(['adult', 'id', 'original_language', 'original_title', 'overview',
       'popularity', 'runtime', 'tagline', 'title', 'vote_average',
       'vote_count', 'genres', 'production_companies', 'production_countries',
       'release_year', 'overview_keywords', 'tags', 'directors', 'characters'],
      dtype='object')

In [15]:
df.sort_values(by='vote_count', ascending=False).head(20)

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,production_companies,production_countries,release_year,overview_keywords,tags,directors,characters
9800,False,27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,148.0,Your mind is the scene of the crime.,Inception,8.1,14075.0,"['Action', 'Thriller', 'Science Fiction', 'Mys...",[],[],2010.0,"['cobb', 'skilled', 'thief', 'commit', 'corpor...","['loss of lover', 'dream', 'kidnapping', 'slee...",['Christopher Nolan'],"['Dom Cobb', 'Arthur', 'Ariadne', 'Eames', 'Sa..."
8335,False,155,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,152.0,Why So Serious?,The Dark Knight,8.3,12269.0,"['Drama', 'Action', 'Crime', 'Thriller']",[],[],2008.0,"['batman', 'raise', 'stake', 'war', 'crime', '...","['dc comics', 'crime fighter', 'secret identit...","['Christopher Nolan', 'Steve Gehrke', 'Nilo Ot...","['Bruce Wayne / Batman', 'Alfred Pennyworth', ..."
9364,False,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,162.0,Enter the World of Pandora.,Avatar,7.2,12114.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",[],[],2009.0,"['nd', 'century', 'paraplegic', 'marine', 'dis...","['culture clash', 'future', 'space war', 'spac...",['James Cameron'],"['Jake Sully', 'Princess Neytiri', 'Dr. Grace ..."
10825,False,24428,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,143.0,Some assembly required.,The Avengers,7.4,12000.0,"['Science Fiction', 'Action', 'Adventure']",[],[],2012.0,"['unexpected', 'enemy', 'emerge', 'threaten', ...","['new york', 'shield', 'marvel comic', 'superh...","['Joss Whedon', 'Trudy Ramirez']","['Tony Stark / Iron Man', 'Steve Rogers / Capt..."
14611,False,293660,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,108.0,Witness the beginning of a happy ending,Deadpool,7.4,11444.0,"['Action', 'Adventure', 'Comedy']",[],[],2016.0,"['deadpool', 'tell', 'origin', 'story', 'speci...","['anti hero', 'mercenary', 'marvel comic', 'su...",['Tim Miller'],"['Wade Wilson / Deadpool', 'Vanessa Carlysle /..."
13039,False,157336,en,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,169.0,Mankind was born on Earth. It was never meant ...,Interstellar,8.1,11187.0,"['Adventure', 'Drama', 'Science Fiction']",[],[],2014.0,"['interstellar', 'chronicle', 'adventure', 'gr...","['saving the world', 'artificial intelligence'...",['Christopher Nolan'],"['Joseph Cooper', 'Murph Cooper', 'Dr. Amelia ..."
11778,False,68718,en,Django Unchained,"With the help of a German bounty hunter, a fre...",19.785025,165.0,"Life, liberty and the pursuit of vengeance.",Django Unchained,7.8,10297.0,"['Drama', 'Western']",[],[],2012.0,"['help', 'german', 'bounty', 'hunter', 'freed'...","['bounty hunter', 'hero', 'plantation', 'socie...","['Quentin Tarantino', 'Martin Kitrosser', 'Wil...","['Django', 'Dr. King Schultz', 'Calvin Candie'..."
13444,False,118340,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",53.291601,121.0,All heroes start somewhere.,Guardians of the Galaxy,7.9,10014.0,"['Action', 'Science Fiction', 'Adventure']",[],[],2014.0,"['light', 'year', 'earth', 'year', 'abduct', '...","['marvel comic', 'spaceship', 'space', 'outer ...",['James Gunn'],"['Peter Quill / Star-Lord', 'Gamora', 'Drax th..."
2192,False,550,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,139.0,Mischief. Mayhem. Soap.,Fight Club,8.3,9678.0,['Drama'],[],[],1999.0,"['tickingtimebomb', 'insomniac', 'slippery', '...","['support group', 'dual identity', 'nihilism',...","['David Fincher', 'Dina Waxman']","['The Narrator', 'Tyler Durden', ""Robert 'Bob'..."
11011,False,70160,en,The Hunger Games,Every year in the ruins of what was once North...,20.031667,142.0,May The Odds Be Ever In Your Favor.,The Hunger Games,6.9,9634.0,"['Science Fiction', 'Adventure', 'Fantasy']",[],[

In [16]:
def compare_multi_field(f1, f2):
    try:
        return len(set(f1).intersection(set(f2))) / len(set(f1).union(set(f2)))
    except:
        return 0

def the_most_similar_movies(movie_id, n=5, w_adult = 0.1, w_original_language = 0.1, w_release_year = 0.1, w_genres = 0.3, w_production_companies = 0.1, w_production_countries = 0.1, w_overview_keyword = 0.3, w_popularity = 0.1, w_tags = 0.3, w_characters = 0.3, w_directors = 0.3):
    movie_1 = df.loc[movie_id]
    print(movie_1["title"], movie_1["genres"])
    similarities = []
    for i, movie_2 in df.iterrows():
        if i == movie_id:
            continue
        adult = 1 if movie_1['adult'] == movie_2['adult'] else 0
        original_language = 1 if movie_1['original_language'] == movie_2['original_language'] else 0
        release_year = 1 - (abs(movie_1['release_year'] - movie_2['release_year']) / 100)
        genres = compare_multi_field(movie_1['genres'], movie_2['genres'])
        production_companies = compare_multi_field(movie_1['production_companies'], movie_2['production_companies'])
        production_countries = compare_multi_field(movie_1['production_countries'], movie_2['production_countries'])
        overview_keywords = compare_multi_field(movie_1['overview_keywords'], movie_2['overview_keywords'])
        tags = compare_multi_field(movie_1['tags'], movie_2['tags'])
        characters = compare_multi_field(movie_1['characters'], movie_2['characters'])
        directors = compare_multi_field(movie_1['directors'], movie_2['directors'])
        try:
            popularity = 1 - (abs(float(movie_1['popularity']) - float(movie_2['popularity'])) / 100)
        except:
            print(movie_1['popularity'], movie_2['popularity'], "!!!!!!!!!!!!!!!")
            popularity = 0
        # popularity = 1 - (abs(movie_1['popularity'] - movie_2['popularity']) / 100)
        # similarity = w_adult * adult +  w_original_language * original_language + w_release_year * release_year + w_genres * genres + w_production_companies * production_companies + w_production_countries * production_countries + w_overview_keyword * overview_keywords + w_popularity * popularity 
        
        similarity = w_adult * adult +  w_original_language * original_language + w_release_year * release_year + w_genres * genres + w_production_companies * production_companies + w_production_countries * production_countries + w_overview_keyword * overview_keywords + w_popularity * popularity + w_tags * tags + w_characters * characters + w_directors * directors
        similarities.append((movie_2["title"], similarity.item(), movie_2["genres"]))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:n]

In [23]:
the_most_similar_movies(234, 5)

Pulp Fiction ['Thriller', 'Crime']


[('Reservoir Dogs', 1.758661204702576, "['Crime', 'Thriller']"),
 ('Snatch', 1.7011016087997775, "['Thriller', 'Crime']"),
 ('Gone Girl', 1.6951467207343358, "['Mystery', 'Thriller', 'Drama']"),
 ('Cape Fear', 1.6770590876699507, "['Crime', 'Thriller']"),
 ('The Dark Knight',
  1.6738052582941179,
  "['Drama', 'Action', 'Crime', 'Thriller']")]

In [18]:
the_most_similar_movies(314, 10)

Mirage ['Action', 'Thriller']


[('Bear', 1.8624522795314402, "['Horror', 'Action', 'Thriller']"),
 ('Caught Inside', 1.8242348514137932, "['Thriller', 'Foreign']"),
 ('The Raven', 1.8212870952961462, "['Horror', 'Thriller']"),
 ('Riding with Death',
  1.7535832966403944,
  "['Science Fiction', 'TV Movie', 'Action', 'Thriller']"),
 ('Gupt: The Hidden Truth',
  1.7498555967749545,
  "['Action', 'Romance', 'Thriller']"),
 ('Morning Star', 1.7393884444822136, "['Fantasy', 'Action', 'Adventure']"),
 ('They Call Me Macho Woman', 1.737961305328336, "['Action']"),
 ('Alien Origin',
  1.732460091952381,
  "['Fantasy', 'Thriller', 'Science Fiction']"),
 ('Patch Town',
  1.7253469226477114,
  "['Music', 'Adventure', 'Comedy', 'Fantasy', 'Science Fiction']"),
 ('Killer Legends',
  1.7240433560188086,
  "['History', 'Crime', 'Horror', 'Documentary']")]

In [19]:
the_most_similar_movies(834, 15)

Delicatessen ['Comedy', 'Science Fiction', 'Fantasy']


[('The City of Lost Children',
  1.7723460029151137,
  "['Fantasy', 'Science Fiction', 'Adventure']"),
 ('The Visitors',
  1.7614796160092165,
  "['Fantasy', 'Comedy', 'Science Fiction']"),
 ('Zazie dans le métro', 1.7263989167476392, "['Comedy', 'Fantasy']"),
 ('Amélie', 1.721533356640394, "['Comedy', 'Romance']"),
 ('The Visitors II: The Corridors of Time',
  1.7104880422603157,
  "['Comedy', 'Fantasy']"),
 ('Renaissance',
  1.7088296296483516,
  "['Action', 'Animation', 'Science Fiction']"),
 ('That Obscure Object of Desire',
  1.6931820796814416,
  "['Comedy', 'Drama', 'Romance']"),
 ('Elf', 1.6921348767392952, "['Comedy', 'Family', 'Fantasy']"),
 ('Just Visiting',
  1.6849491566267103,
  "['Comedy', 'Fantasy', 'Science Fiction']"),
 ('Doc Hollywood', 1.6836250036219336, "['Comedy', 'Romance']"),
 ('Micmacs', 1.6828133086967534, "['Action', 'Comedy', 'Crime']"),
 ('A Town Called Panic',
  1.6824407622467534,
  "['Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy']"),
 ('Rififi'

In [21]:
the_most_similar_movies(3670, 15)

The Lord of the Rings: The Fellowship of the Ring ['Adventure', 'Fantasy', 'Action']


[('The Lord of the Rings: The Return of the King',
  1.9849064107777776,
  "['Adventure', 'Fantasy', 'Action']"),
 ('The Hobbit: The Battle of the Five Armies',
  1.9836006909183546,
  "['Action', 'Adventure', 'Fantasy']"),
 ('The Lord of the Rings: The Two Towers',
  1.9745508639480518,
  "['Adventure', 'Fantasy', 'Action']"),
 ('The Hobbit: An Unexpected Journey',
  1.8039617157477967,
  "['Adventure', 'Fantasy', 'Action']"),
 ("Pirates of the Caribbean: Dead Man's Chest",
  1.7817971829849624,
  "['Adventure', 'Fantasy', 'Action']"),
 ('Harry Potter and the Prisoner of Azkaban',
  1.781372380268613,
  "['Adventure', 'Fantasy', 'Family']"),
 ('Pirates of the Caribbean: On Stranger Tides',
  1.7780114394444444,
  "['Adventure', 'Action', 'Fantasy']"),
 ('Krull', 1.7767578441048952, "['Fantasy', 'Action', 'Adventure']"),
 ('The Amazing Spider-Man',
  1.7752703643261252,
  "['Action', 'Adventure', 'Fantasy']"),
 ("Frank Herbert's Dune",
  1.771736689452637,
  "['Action', 'Adventure', 'F